#TP4(Projet)

#imports

In [ ]:
import time
import os
import yake
from collections import Counter
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image
import sys
import re
from nltk.tokenize import sent_tokenize
import spacy
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode
import collections
import string
import pandas as pd
from nltk import word_tokenize
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
import nltk
from nltk.tokenize import sent_tokenize
import shutil

In [ ]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [ ]:
# Liste de Stopwords enrichis (Idem que dans s1)
sw = stopwords.words("french")
sw += ["abord","deux","tout", "ailleurs", "ainsi", "alors", "après", "avant", "avoir", "bien", "bientôt",
    "car", "ce", "ceci", "cela", "ces", "cet", "cette", "comme", "contre", "dans",
    "depuis", "dire", "doit", "donc", "elle", "encore", "enfin", "ensuite", "entre", "être",
    "fait", "faire", "faut", "hormis", "ici", "il", "ils", "jusqu'à", "les", "leur", "là",
    "ma", "mais", "mes", "moins", "mon", "ne", "non", "nous", "on", "ou", "ou bien", "par",
    "parce que", "parfois", "pas", "pendant", "peut", "plus", "plutôt", "pour", "puis",
    "quand", "quant à", "sans", "sa", "se", "ses", "son", "sous", "tant pis", "tandis que",
    "trois", "tôt ou tard", "toutefois", "toutes", "tu", "tous", "trois", "trois", "trois",
    "voilà", "à", "très"]
sw = set(sw)

In [ ]:
data_path = "../data/txttp4/"

#Chemin vers les differents fichiers de notre travail 

In [ ]:
files = [f for f in sorted(os.listdir(data_path)) if f.endswith('.txt')]

In [ ]:
# taille des fichiers
files
len(files)

In [ ]:
# Exemple de fichiers
files[:5]

In [ ]:
# Stocker le contenu de ces fichiers dans une liste en utilisant la compréhension de liste
texts = [open(data_path + f, "r", encoding="utf-8").read() for f in files]

In [ ]:
#for text in texts:
    #for char in text:
       # print(char, end='', flush=True)
        #time.sleep(0.01)

In [ ]:

# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'texts.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(texts))


In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'texts.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

In [ ]:
def clean_text(folder=None):
    if folder is None:
        input_path = f"texts.txt"
        output_path = f"texts_clean.txt"
    else:
        input_path = f"{folder}/texts.txt"
        output_path = f"{folder}/texts_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text( folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'texts_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

#  Extraire uniquement les phrases "pertinentes" dans les documents

In [ ]:
query = ["volley", "volleyball","volley-ball"]


In [ ]:
# Création d'une regex afin de trouver les mots de la liste query dans le texte
regex = re.compile(f"\\b({'|'.join(query)})\\b", re.IGNORECASE)

In [ ]:
# Le répertoire qui contient vos fichiers txt exportés de Camille
indir = "../data/tmp/"
# Le répertoire qui contiendra les fichiers txt nettoyés
outdir = "../data/tmp/regex/"

if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
for file in os.listdir(indir)[:10]:
    if file.endswith("texts_clean.txt"):
        relevant_sentences = []
        f_in = open(os.path.join(indir, file), encoding="utf-8")
        text = f_in.read()
        for sentence in sent_tokenize(text):
            if regex.search(sentence):
                relevant_sentences.append(sentence)
        f_in.close()
        f_out = open(os.path.join(outdir, file), "w", encoding="utf-8")
        f_out.write("\n\n".join(relevant_sentences))
        f_out.close()

In [ ]:
# Vérifier le résultat
with open(os.path.join(outdir, f'texts_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

### Afficher les termes les plus fréquents


In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(20))

# Nuage de mots

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"texts.png"))
Image(filename=os.path.join(temp_path, f"texts.png"))

In [ ]:
this_file = files
this_file

In [ ]:
# Initialisation de liste pour stocker les textes
texts_n = []

# Parcourir la liste des noms de fichiers
for this_file in this_file:
    file_path = os.path.join(data_path, this_file)
    
    # Vérifier si le fichier existe
    if os.path.exists(file_path):
        # Lecture du contenu du fichier et stockage dans la liste
        with open(file_path, 'r', encoding='utf-8') as file:
            file_text = file.read()
            texts_n.append(file_text)

In [ ]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(file_text)
keywords

In [ ]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

# Evolution de thematiques de volleyball au fil des annees 

## Deplacement des fichiers dans les dossiers correspondant a chaque annee 

# important de faire une copie du repertoire principal de fichier avant  de deplacer les fichiers

In [ ]:


# Chemin vers le répertoire contenant les fichiers
data_path = "../data/txttp4/"

# Créer un répertoire pour chaque année entre 1830 et 1970
for year in range(1830, 1971):
    year_folder = f"../data/txttp4/{year}"
    os.makedirs(year_folder, exist_ok=True)

# Liste des fichiers dans le répertoire principal
files = os.listdir(data_path)

# Déplacer les fichiers vers les sous-répertoires correspondants
for file in files:
    # Extraire l'année du nom de fichier
    try:
        year = int(file.split('_')[2].split('-')[0])
    except (ValueError, IndexError):
        # Ignorer les fichiers qui ne suivent pas le format attendu
        continue

    # Afficher le chemin du fichier et le chemin de destination
    source_path = os.path.join(data_path, file)
    destination_path = os.path.join(data_path, str(year), file)
    print(f"Moving {file} from {source_path} to {destination_path}")

    try:
        # Déplacer le fichier vers le sous-répertoire correspondant à l'année
        shutil.move(source_path, destination_path)
        print(f"Successfully moved {file}")
    except Exception as e:
        print(f"Error moving {file}: {e}")


In [ ]:
import os
import matplotlib.pyplot as plt

# Chemin vers le répertoire principal
data_path = "../data/txttp4/"

# Liste des années
years = list(range(1830, 1971))

# Nombre de fichiers par année
file_counts = []

# Parcourir les années et compter le nombre de fichiers dans chaque dossier
for year in years:
    year_folder = os.path.join(data_path, str(year))
    if os.path.exists(year_folder):
        files_in_year = len(os.listdir(year_folder))
        file_counts.append(files_in_year)
    else:
        file_counts.append(0)

# Tracer le graphique
plt.plot(years, file_counts, marker='o')
plt.title('Nombre de fichiers par année')
plt.xlabel('Année')
plt.ylabel('Nombre de fichiers')
plt.grid(True)
plt.show()


# Etude des corpus en fonctions des intervalles d'annees ou annees

In [ ]:
import os

# Chemin vers le répertoire principal
data_path = "../data/txttp4/"

# Intervalles d'années que vous souhaitez extraire
start_year = 1941
end_year = 1955  

# Liste pour stocker les noms de fichiers
N_files = []

# Parcourir les années dans l'intervalle spécifié
for year in range(start_year, end_year + 1):  # +1 pour inclure également end_year
    year_folder = os.path.join(data_path, str(year))
    
    # Ajouter les fichiers se terminant par '.txt' dans la liste
    N_files.extend([os.path.join(year_folder, f) for f in sorted(os.listdir(year_folder)) if f.endswith('.txt')])

# Afficher les noms de fichiers
print(N_files)




In [ ]:
# Nombre de fichiers 
len(N_files)

In [ ]:
# Stocker le contenu de ces fichiers dans une liste en utilisant la compréhension de liste
N_texts = [open(f, "r", encoding="utf-8").read() for f in N_files]

In [ ]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'N_texts.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(N_texts))

In [ ]:
# Imprimer le contenu du fichier et constater les "déchets"
with open(os.path.join(temp_path, f'N_texts.txt'), 'r', encoding='utf-8') as f:
    before = f.read()

before[:500]

In [ ]:
def clean_text(folder=None):
    if folder is None:
        input_path = f"N_texts.txt"
        output_path = f"N_texts_clean.txt"
    else:
        input_path = f"{folder}/N_texts.txt"
        output_path = f"{folder}/N_texts_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text( folder=temp_path)

In [ ]:
# Vérifier le résultat
with open(os.path.join(temp_path, f'texts_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

In [ ]:
query = ["volley", "volleyball","volley-ball"]

In [ ]:
# Création d'une regex afin de trouver les mots de la liste query dans le texte
regex = re.compile(f"\\b({'|'.join(query)})\\b", re.IGNORECASE)

In [ ]:
# Le répertoire qui contient vos fichiers txt exportés de Camille
indir = "../data/tmp/"
# Le répertoire qui contiendra les fichiers txt nettoyés
outdir = "../data/tmp/regex/"

if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
for file in os.listdir(indir)[:10]:
    if file.endswith("N_texts_clean.txt"):
        relevant_sentences = []
        f_in = open(os.path.join(indir, file), encoding="utf-8")
        text = f_in.read()
        for sentence in sent_tokenize(text):
            if regex.search(sentence):
                relevant_sentences.append(sentence)
        f_in.close()
        f_out = open(os.path.join(outdir, file), "w", encoding="utf-8")
        f_out.write("\n\n".join(relevant_sentences))
        f_out.close()

In [ ]:
# Vérifier le résultat
with open(os.path.join(outdir, f'N_texts_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

In [ ]:
frequencies = Counter(after.split())
print(frequencies.most_common(200))

In [ ]:
cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"N_texts.png"))
Image(filename=os.path.join(temp_path, f"N_texts.png"))

# Les mots clés de mon sous corpus 

In [ ]:
from rake_nltk import Rake

# Charger le contenu du fichier Texts_clean
with open("../data/tmp/regex/N_texts_clean.txt", 'r', encoding="utf-8") as file:
    text = file.read()

# Initialiser l'extracteur de mots-clés Rake avec des paramètres personnalisés
kw_extractor = Rake(min_length=2, max_length=5)  # Vous pouvez ajuster ces valeurs selon vos besoins

# Extraire les mots-clés du texte
kw_extractor.extract_keywords_from_text(text)

# Récupérer les mots-clés
keywords = kw_extractor.get_ranked_phrases()

# Filtrer les mots-clés pour ne conserver que ceux de deux mots
kept = [kw for kw in keywords if len(kw.split()) == 2]

# Afficher les mots-clés conservés
print(f"Mots-clés conservés : {', '.join(kept)}")

# Les noms des personnes et des endroits pour voir comment les acteurs principaux du volleyball en Belgique ont évolué

In [ ]:
import spacy
from collections import defaultdict
from multiprocessing import Pool

# Initialiser le modèle SpaCy pour le français
nlp = spacy.load("fr_core_news_sm")

# Charger le texte
n = 100000
text = open("../data/tmp/N_texts.txt", encoding='utf-8').read()[:n]

# Traiter le texte
doc = nlp(text)

# Compter les entités
people = defaultdict(int)
locations = defaultdict(int)

for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1
    elif ent.label_ == "LOC":
        locations[ent.text] += 1

# Trier et imprimer les personnes
sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)
print("\nPersonnes les plus mentionnées :")
for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

# Trier et imprimer les lieux
sorted_locations = sorted(locations.items(), key=lambda kv: kv[1], reverse=True)
print("\nLieux les plus mentionnés :")
for location, freq in sorted_locations[:50]:
    print(f"{location} apparait {freq} fois dans le corpus")





# Segmentation en phrases

## Fichiers d'inputs et d'outputs

In [ ]:
infile = "../data/tmp/texts.txt"
outfile = "../data/tmp/regex/sen.txt"

## Segmentation en phrases du corpus complet et création d'un nouveau fichier

In [ ]:
LIMIT = 1000000

In [ ]:
with open(outfile, 'w', encoding="utf-8") as output:
    with open(infile, encoding="utf-8", errors="backslashreplace") as f:
        content = f.readlines()
        content = content[:LIMIT] if LIMIT is not None else content
        n_lines = len(content)
        for i, line in enumerate(content):
            if i % 100 == 0:
                print(f'processing line {i}/{n_lines}')
            sentences = sent_tokenize(line)
            for sent in sentences:
                output.write(sent + "\n")
print("Done")

# Word_Embedding

# crreation d'un objet streame

In [ ]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [ ]:
infile = f"../data/tmp/regex/sen.txt"
sentences = MySentences(infile)

# Detection des bigrams

In [ ]:
bigram_phrases = Phrases(sentences)

In [ ]:
type(bigram_phrases.vocab)

In [ ]:
len(bigram_phrases.vocab.keys())

In [ ]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

In [ ]:
bigram_phrases.vocab[key_]

# conversion des Phrases en objet Phraser

In [ ]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

# Extraction trigrams

In [ ]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [ ]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

# creation corpus avec unigrams, bi , tri 

In [ ]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [ ]:
print(corpus[:10])

In [ ]:
# Définir différentes valeurs pour window et min_count
window_values = [3, 5, 7]
min_count_values = [3, 5, 10]

# Des essais-erreurs
best_model = None
best_score = float('-0.01')  

for window_size in window_values:
    for min_count_value in min_count_values:
        print(f"Entraînement avec window={window_size}, min_count={min_count_value}")
        
        # Entraînement du modèle
        model = Word2Vec(
            corpus,
            vector_size=32,
            window=window_size,
            min_count=min_count_value,
            workers=4,
            epochs=5
        )

        score = len(model.wv.key_to_index)

        print(f"Score obtenu : {score}\n")

        # Comparaison avec le meilleur modèle précédent
        if score > best_score:
            best_score = score
            best_model = model

print("Entraînement terminé.")


In [ ]:
outfile = f"../data/newspapers_n.model"
model.save(outfile)

In [ ]:
model = Word2Vec.load("../data/newspapers_n.model")

In [ ]:
model.wv["volleyball"]

In [ ]:
model.wv.similarity("sc", "vainqueurs")

In [ ]:
model.wv.most_similar("royal", topn=15)

In [ ]:
model.wv.most_similar("champions", topn=10)

In [ ]:
print(model.wv.most_similar(positive=['volleyball', 'joueur'], negative=['equipe']))

In [ ]:
similar_words = model.wv.most_similar("volleyball", topn=5)

# Affichez les mots similaires
for word, score in similar_words:
    print(word, score)

In [ ]:
print(model.wv.most_similar(positive=['volleyball', 'plage']))
print("\n")

In [ ]:
# Recherche de termes liés au volleyball en compétition
print("Termes liés au volleyball en compétition:")
print(model.wv.most_similar(positive=['volleyball', 'competition']))
print("\n")

In [ ]:
# Trouver des termes associés au volleyball comme divertissement plutôt que fatigue 
print("Termes associés au volleyball comme divertissement:")
print(model.wv.most_similar(positive=['volleyball', 'divertissement'], negative=['fatigue']))
print("\n")

In [ ]:

# Exploration des attributs de joueur, par exemple, la puissance dans le contexte du volleyball
print("Exploration des attributs de joueur - Puissant vs Rapide:")
print(model.wv.most_similar(positive=['volleyball', 'puissant'], negative=['rapide']))